In [2]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

We're going to first try training a CNN on the individual images.
We will be using binary cross entropy across the 17 regions.

In [2]:
BATCH_SIZE = 20
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17

class Sequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        
        order = np.arange(self.num_batches)
        np.random.seed(0)
        np.random.shuffle(order)
        k = 0
        for k in range(len(order)):
            self.idx_dict[k]=order[k]

    def __len__(self):
        return self.num_batches
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        idx = self.idx_dict[idx] #Mix up order of the batches 
        
        path = "temp/{}/batch_{}.hdf5".format(self.mode,idx)
        key = "{}/batch_{}.hdf5".format(self.mode,idx)
        bucket.download_file(Key=key,Filename=path)
        
        f = h5py.File(path,"r")
        try:
            return f['/image'].value, f['/labels'].value
        finally:
            f.close()
            os.remove(path)        



Let's see how an existing architecture would do. 

In [8]:
from inception_resnet_v1 import InceptionResNetV1
from keras.layers import Input,Flatten,Dense,Concatenate
from keras.models import Model
import keras


input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
incep = InceptionResNetV1(include_top=False,
                          weights=None,
                          input_tensor=input_img,
                         pooling='max')
last = incep.output
#last = Flatten()(last)

#List of independent guesses for each zone
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(last))

out = keras.layers.concatenate(output_nodes)

multi_label_model = Model(input_img, out)




from keras.layers import Input,Flatten
from keras.applications.inception_resnet_v2 import InceptionResNetV2

input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
resnet = InceptionResNetV2(include_top=False,weights='imagenet',input_tensor=input_img,
                           input_shape =(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))

last = Flatten(resnet)
#List of independent guesses for each zone
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(last))

out = keras.layers.concatenate(output_nodes)
resnet_multi_label_model = Model(input_img, out)

In [11]:
from datetime import datetime
from keras.callbacks import TensorBoard
from keras.optimizers import SGD
from keras import metrics

#Set up learning

#Optimization
lr=0.0005 
momentum=0.5
nes = True
description = "SGD_globalMaxPooling_noDropout"

sgd = SGD(lr,momentum=momentum,nesterov=nes)
multi_label_model.compile(optimizer=sgd,
                          metrics=[metrics.binary_accuracy],
                         loss= 'binary_crossentropy')

#Tensorboard
x = datetime.today()
stamp = "{}-{}-{}_{}:{}:{}_lr-{}_b1-{}_b2-{}_{}".format(x.year,x.month,
                                                     x.day,x.hour,x.minute,
                                                     x.second,lr,momentum,nes,description)
tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                          write_grads=True,write_images=True)



#Generators and fit

#Bucket with clean data
UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
key_id, secret_key = hfuncs.GetAWSCredentials()
client = hfuncs.GetAWSClient(key_id,secret_key)
bucket = client.Bucket(UPLOAD_BUCKET)

#Initialize train sequencer
mode ="train"
num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
train_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

#Initialize validation sequencer
mode = "val"
num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
val_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)


hist = multi_label_model.fit_generator(train_seq,
                                       steps_per_epoch=20,
                                       epochs=20,
                                       validation_data = val_seq,
                                       validation_steps = 140,
                                       callbacks=[tensorboard],
                                      use_multiprocessing =True,workers=4)


Epoch 1/20
20/20 [==============================] - 77s - loss: 0.6646 - binary_accuracy: 0.8016 - val_loss: 0.5904 - val_binary_accuracy: 0.7991
Epoch 2/20
20/20 [==============================] - 66s - loss: 0.3926 - binary_accuracy: 0.8959 - val_loss: 0.6007 - val_binary_accuracy: 0.6946
Epoch 3/20
20/20 [==============================] - 123s - loss: 0.4140 - binary_accuracy: 0.8924 - val_loss: 0.6176 - val_binary_accuracy: 0.6391
Epoch 4/20
20/20 [==============================] - 65s - loss: 0.3947 - binary_accuracy: 0.8903 - val_loss: 0.6328 - val_binary_accuracy: 0.6363
Epoch 5/20
20/20 [==============================] - 65s - loss: 0.3784 - binary_accuracy: 0.8976 - val_loss: 0.6227 - val_binary_accuracy: 0.6632
Epoch 6/20
20/20 [==============================] - 67s - loss: 0.3836 - binary_accuracy: 0.8946 - val_loss: 0.5991 - val_binary_accuracy: 0.6526
Epoch 7/20
20/20 [==============================] - 66s - loss: 0.3717 - binary_accuracy: 0.8947 - val_loss: 0.5663 - val_b

In [3]:
from inception_resnet_v1 import InceptionResNetV1
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout
from keras.models import Model
import keras
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import SGD
from keras import metrics

def trainCNNRes(alpha,lr,momentum,pooling,dropout_rate,description=""):
    
    #ResNetv1
    input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    incep = InceptionResNetV1(include_top=False,
                              weights=None,
                              input_tensor=input_img,
                             pooling=pooling,
                             alpha=alpha)
    #Apply dropout specified with specified rate
    last = incep.output
    last = Dropout(dropout_rate,seed=0)(last)
    
    #List of independent guesses for each zone
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(last))

    out = keras.layers.concatenate(output_nodes)
    
    #FInish model
    multi_label_model = Model(input_img, out)

    
    #optimizer
    nes=True
    sgd = SGD(lr,momentum=momentum,nesterov=nes)
    multi_label_model.compile(optimizer=sgd,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_mom-{}_alpha-{}_pooling-{}_dropout-{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,momentum,alpha,pooling,dropout_rate)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=True)

    #Generators and fit

    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='loss',min_delta=0.005,patience=3)
    
    hist = multi_label_model.fit_generator(train_seq,
                                           steps_per_epoch=40,
                                           epochs=10,
                                           validation_data = val_seq,
                                           validation_steps = 140,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=4)
    return hist,multi_label_model

In [7]:
from inception_resnet_v1 import InceptionResNetV1
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout
from keras.models import Model
import keras
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping
from keras.optimizers import SGD,Adam
from keras import metrics

def trainCNNResAdam(alpha,lr,beta1,beta2,pooling,dropout_rate,description=""):
    
    #ResNetv1
    input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    incep = InceptionResNetV1(include_top=False,
                              weights=None,
                              input_tensor=input_img,
                             pooling=pooling,
                             alpha=alpha)
    #Apply dropout specified with specified rate
    last = incep.output
    last = Dropout(dropout_rate,seed=0)(last)
    
    #List of independent guesses for each zone
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(last))

    out = keras.layers.concatenate(output_nodes)
    
    #FInish model
    multi_label_model = Model(input_img, out)

    
    #optimizer
    adam = Adam(lr,beta_1=beta1,beta_2=beta2)
    multi_label_model.compile(optimizer=adam,
                              metrics=[metrics.binary_accuracy],
                             loss= 'binary_crossentropy')

    #Tensorboard
    x = datetime.today()
    stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_alpha-{}_pooling-{}_dropout-{}".format(x.year,x.month,
                                                         x.day,x.hour,x.minute,
                                                         x.second,lr,beta1,beta2,alpha,pooling,dropout_rate)
    tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                              write_grads=False,write_images=True)

    #Generators and fit

    #Bucket with clean data
    UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
    key_id, secret_key = hfuncs.GetAWSCredentials()
    client = hfuncs.GetAWSClient(key_id,secret_key)
    bucket = client.Bucket(UPLOAD_BUCKET)

    #Initialize train sequencer
    mode ="train"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    train_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Initialize validation sequencer
    mode = "val"
    num_batches = sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1 #train,test,val root directories have their own keys
    val_seq = Sequencer(num_batches,UPLOAD_BUCKET,mode=mode)

    #Early stopping callback
    estop = EarlyStopping(monitor='loss',min_delta=0.005,patience=3)
    
    hist = multi_label_model.fit_generator(train_seq,
                                           steps_per_epoch=100,
                                           epochs=21,
                                           validation_data = val_seq,
                                           validation_steps = 140,
                                           callbacks=[tensorboard,estop],
                                          use_multiprocessing =True,workers=4)
    return hist,multi_label_model

Hypermarameter Oprimization (Later)

from scipy import stats
from sklearn.model_selection import ParameterSampler
import pickle

lower=-1
upper=-4
mu =-2.5
sigma = 0.5

log_lr = stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)

lower=0
upper=1
mu =0.5
sigma =0.10

momentum= stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)

alpha=[2]

pooling = ['max','average']

lower=0
upper=0.5
mu =0
sigma =0.20

dropout = stats.truncnorm((lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma)

param_grid = {'log_lr':log_lr,'momentum':momentum,
             'alpha':alpha,'pooling':pooling,
             'dropout':dropout}
param_list = list(ParameterSampler(param_grid,n_iter=25,random_state=0))

rounded_list=[]
for d in param_list:
    dropout = np.around(d['dropout'],decimals=2)
    momentum = np.around(d['momentum'],decimals=2)
    lr = np.around(10**d['log_lr'],decimals=4)
    pooling = d['pooling']
    alpha = d['alpha']
    rounded_list.append({'lr':lr,
                         'momentum':momentum,
                         'alpha':alpha,
                         'pooling':pooling,
                         'dropout':dropout})
with open("param_list.pickle","wb") as f:
    pickle.dump(rounded_list,f)

#25 Models
hist_list = []
for d in rounded_list:
    try:
        hist_list.append(trainCNNRes(alpha=d['alpha'],
                                     lr=d['lr'],
                                     momentum=d['momentum'],
                                     pooling=d['pooling'],
                                     dropout_rate=d['dropout'],
                                     description=""))
    except:
        print("Something went wrong")
        hist_list.append("Failure")
        


In [4]:
hist,model = trainCNNRes(alpha=1,lr=0.0001,momentum=0.5,pooling='max',dropout_rate=0.20,description="")

Epoch 1/10
40/40 [==============================] - 91s - loss: 1.2085 - binary_accuracy: 0.7030 - val_loss: 0.5163 - val_binary_accuracy: 0.8268
Epoch 2/10
40/40 [==============================] - 84s - loss: 0.7333 - binary_accuracy: 0.7988 - val_loss: 0.4895 - val_binary_accuracy: 0.8546
Epoch 3/10
40/40 [==============================] - 84s - loss: 0.6391 - binary_accuracy: 0.8389 - val_loss: 0.4499 - val_binary_accuracy: 0.8863
Epoch 4/10
40/40 [==============================] - 86s - loss: 0.6201 - binary_accuracy: 0.8460 - val_loss: 0.4067 - val_binary_accuracy: 0.8947
Epoch 5/10
40/40 [==============================] - 82s - loss: 0.5995 - binary_accuracy: 0.8522 - val_loss: 0.3717 - val_binary_accuracy: 0.8967
Epoch 6/10
40/40 [==============================] - 82s - loss: 0.6309 - binary_accuracy: 0.8483 - val_loss: 0.3550 - val_binary_accuracy: 0.8978
Epoch 7/10
40/40 [==============================] - 82s - loss: 0.5865 - binary_accuracy: 0.8576 - val_loss: 0.3593 - val_bi

In [8]:
del hist,model
hist,model = trainCNNResAdam(alpha=1,lr=0.0001,beta1=0.9,beta2=0.99,pooling='max',dropout_rate=0.20,description="")

Epoch 1/21
100/100 [==============================] - 148s - loss: 0.6179 - binary_accuracy: 0.8548 - val_loss: 0.4634 - val_binary_accuracy: 0.8359
Epoch 2/21
100/100 [==============================] - 164s - loss: 0.4973 - binary_accuracy: 0.8647 - val_loss: 0.3930 - val_binary_accuracy: 0.8790
Epoch 3/21
100/100 [==============================] - 142s - loss: 0.4563 - binary_accuracy: 0.8708 - val_loss: 0.3865 - val_binary_accuracy: 0.8984
Epoch 4/21
100/100 [==============================] - 142s - loss: 0.4104 - binary_accuracy: 0.8840 - val_loss: 0.3959 - val_binary_accuracy: 0.8984
Epoch 5/21
100/100 [==============================] - 142s - loss: 0.3990 - binary_accuracy: 0.8856 - val_loss: 0.3936 - val_binary_accuracy: 0.8982
Epoch 6/21
100/100 [==============================] - 142s - loss: 0.3819 - binary_accuracy: 0.8880 - val_loss: 0.3800 - val_binary_accuracy: 0.8984
Epoch 7/21
100/100 [==============================] - 141s - loss: 0.3612 - binary_accuracy: 0.8951 - val_

In [9]:
import h5py

#Save trained model
model.save("Final_CNNRes.h5")


In [22]:
print(model.layers[-20].get_config())
print(model.get_layer(name='global_max_pooling2d_3').get_config())

{'data_format': 'channels_last', 'name': 'global_max_pooling2d_3', 'trainable': True}
{'data_format': 'channels_last', 'name': 'global_max_pooling2d_3', 'trainable': True}


In [23]:
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout

#Number of angles per scan 
ANGLES = 32

#Hidden dimensions
LSTM_OUTPUT_DIM = 1000

#Build new sequential model,Removing dense layers from model
input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
sequenced_model = TimeDistributed(model.get_layer(name='global_max_pooling2d_3'))(input_scan)

#One lstm layer for now
lstm = LSTM(LSTM_OUTPUT_DIM)(sequenced_model)

#Finally, 17 dense layers connected to the output
output_nodes = []
for i in range(ZONES):
    output_nodes.append(Dense(1,activation='sigmoid')(lstm))

out = keras.layers.concatenate(output_nodes)

#complete model
multi_label_model = Model(input_scan, out)




